In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [2]:
def cohen_glass_hedges(x,y):
    nx = len(x)
    ny = len(y)
    pstdnmr = (nx-1)*x.var(ddof=1) + (ny-1)*y.var(ddof=1)   # (n-1)(var1+var2)
    pstddnm = nx + ny - 2                                   # (n-1)*2
    pstd = (pstdnmr / pstddnm)**(0.5)
    cohen = (x.mean() - y.mean()) / pstd
    glass = (x.mean() - y.mean()) / x.std(ddof=1)
    hedges = cohen * (1 - 3/(4 * (nx+ny) - 9) )             # cohen * (1-3/(8*n-9))
    return (cohen, glass, hedges)

In [3]:
am = pd.read_csv('aeroscan_full_melt.csv')
ap = pd.read_csv('aeroscan_full_pivot.csv')

In [4]:
am.head(3)

,ID,sex,location,time,level,parameter,value
0,t1,f,Jurilovca,f,1,rrate,20.690417
1,t2,m,Baia,f,1,rrate,38.483108
2,t3,f,Jurilovca,f,1,rrate,34.786918


In [5]:
ap.head(3)

,ID,sex,location,time,level,rrate,hrate,ventilation,vo2,vo2rel,...,vco2rel,rer,vo2eq,vo2pulse,fat,carb,fat100,carb100,energy,met
0,t1,f,Jurilovca,f,1,20.690417,142.380702,30.818651,1.027298,14.675693,...,16.311019,1.111431,29.999704,7.215153,0.0,312.105589,0.0,100.0,312.105589,4.193055
1,t1,f,Jurilovca,f,2,19.582856,144.350852,30.663501,1.021342,14.590605,...,17.041474,1.167976,30.022744,7.075416,0.0,310.296048,0.0,100.0,310.296048,4.168744
2,t1,f,Jurilovca,f,3,20.252536,156.988301,32.974861,1.071063,15.300896,...,18.420167,1.203862,30.787048,6.822564,0.0,325.401693,0.0,100.0,325.401693,4.371685


In [6]:
aml = am.copy()

In [7]:
aml.parameter.unique()

array(['rrate', 'hrate', 'ventilation', 'vo2', 'vo2rel', 'vco2',
       'vco2rel', 'rer', 'vo2eq', 'vo2pulse', 'fat', 'carb', 'fat100',
       'carb100', 'energy', 'met'], dtype=object)

In [8]:
aml.parameter = aml.parameter.replace(dict(zip(['rrate', 'hrate',
                                                'ventilation', 'vo2', 'vo2rel',
                                                'vco2', 'vco2rel',
                                                'rer', 'vo2eq', 'vo2pulse',
                                                'fat', 'carb',
                                                'fat100', 'carb100', 
                                                'energy', 'met'],
              ['Respiratory rate (breaths/min)', 'Heart rate (beats/min)', 'Ventilation (l/min)', 
               'O2 intake (l/min)', 'Relative O2 intake (ml/kg/min)',
               'CO2 expired (l/min)', 'Relative CO2 expired (ml/kg/min)',
               'Respiratory exchange ratio', 'Ventilation oxygen equivalent (l)', 'O2 per pulse (l)',
               'Fat energy (kcl/h)', 'Carbohydrates energy (kcal/h)',
               'Fat energy %', 'Carbohydrates energy %',
               'Total energy (kcal/h)', 'Metabolic equivalent of task (MET) (3.5ml/kg/min)'])))

In [9]:
aml.time = aml.time.replace(dict(zip(['f','i'],['final', 'initial'])))

In [10]:
aml.columns

Index(['ID', 'sex', 'location', 'time', 'level', 'parameter', 'value'], dtype='object')

In [11]:
aml.columns = ['ID', 'sex', 'Location', 'Time', 'Effort level', 'parameter', 'value']

In [12]:
apl = ap.copy()

In [13]:
apl.columns

Index(['ID', 'sex', 'location', 'time', 'level', 'rrate', 'hrate',
       'ventilation', 'vo2', 'vo2rel', 'vco2', 'vco2rel', 'rer', 'vo2eq',
       'vo2pulse', 'fat', 'carb', 'fat100', 'carb100', 'energy', 'met'],
      dtype='object')

In [14]:
apl.columns = ['ID', 'sex', 'Location', 'Time', 'Effort level',
               'Respiratory rate (breaths/min)', 'Heart rate (beats/min)', 'Ventilation (l/min)', 
               'O2 intake (l/min)', 'Relative O2 intake (ml/kg/min)',
               'CO2 expired (l/min)', 'Relative CO2 expired (ml/kg/min)',
               'Respiratory exchange ratio', 'Ventilation oxygen equivalent (l)', 'O2 per pulse (l)',
               'Fat energy (kcal/h)', 'Carbohydrates energy (kcal/h)',
               'Fat energy %', 'Carbohydrates energy %',
               'Total energy (kcal/h)', 'Metabolic equivalent of task (MET) (3.5ml/kg/min)']

In [15]:
apl.Time = apl.Time.replace(dict(zip(['f','i'],['final', 'initial'])))

In [16]:
# sns.catplot(data=aml,
#             x='Effort level', y='value',
#             hue='Time', hue_order=['initial','final'],
#             col='parameter', row='sex',
#             sharex=False, sharey=False, 
#             kind='point',
#             dodge=True, linewidth=3, estimator='mean', 
#             errorbar='se')

In [17]:
# for col in apl.columns[5:]:
#     mpl.pyplot.figure(figsize=(8,6))
#     sns.set_style("whitegrid")
#     ax = sns.violinplot(data=apl,
#                         x='Effort level', y=col,
#                         hue='Time', hue_order=['initial','final'],
#                         split=False, inner='box')
#     sns.move_legend(ax, "lower right")
#    plt.savefig(f'figures/{col.replace("/"," ")}-violin.pdf')
#    plt.savefig(f'figures/{col.replace("/"," ")}-violin.svg')

In [18]:
# for param in aml.parameter.unique():
#     mpl.pyplot.figure(figsize=(14,6))
#     sns.set_style("whitegrid")
#     sns.catplot(data=aml.loc[aml.parameter==param],
#                 x='Effort level', y='value',
#                 hue='Time', hue_order=['initial','final'],
#                 col='sex',
#                 sharex=False, sharey=True,
#                 kind='point',
#                 dodge=True, linewidth=3, estimator='mean',
#                 errorbar='se').set_ylabels(param)
#    plt.savefig(f'figures/point/{param.replace("/"," ")}-pointplot.pdf')
#    plt.savefig(f'figures/point/{param.replace("/"," ")}-pointplot.svg')

In [19]:
# for param in aml.parameter.unique()[10:14]:
#     mpl.pyplot.figure(figsize=(14,6))
#     sns.set_style("whitegrid")
#     sns.catplot(data=aml.loc[aml.parameter==param],
#                 x='Effort level', y='value',
#                 hue='Time', hue_order=['initial','final'],
#                 col='sex',
#                 sharex=False, sharey=True,
#                 kind='strip',
#                 dodge=True, linewidth=0.5, estimator='mean',
#                 errorbar='se').set_ylabels(param)
#    plt.savefig(f'figures/strip/{param.replace("/"," ")}-stripplot.pdf')
#    plt.savefig(f'figures/strip/{param.replace("/"," ")}-stripplot.svg')

In [26]:
f=apl[['ID', 'sex', 'Effort level', 'Time', 'Fat energy (kcal/h)']]

In [27]:
g=f.groupby(['sex', 'Effort level', 'Time'])['Fat energy (kcal/h)'].agg(
    lambda x: x.map(lambda y: y==0).sum()).to_frame().reset_index()
g.columns = ['sex', 'Effort level', 'Time', 'Carbohydrate-exclusive metabolism']

In [29]:
# mpl.pyplot.figure(figsize=(14,6))
# sns.set_style("whitegrid")
# sns.catplot(data=g, x='Effort level', y='Carbohydrate-exclusive metabolism',
#             hue='Time', hue_order=['initial','final'],
#             col='sex', sharey=True, kind='bar')
# plt.savefig('figures/rer-above-1-sex-barplot.pdf')
# plt.savefig('figures/rer-above-1-sex-barplot.svg')

In [30]:
for i in ap.columns[20:]:
    print(i, AnovaRM(data=ap, depvar=i, subject='ID', within=['time','level']).fit())

met                  Anova
           F Value Num DF  Den DF Pr > F
----------------------------------------
time        4.4863 1.0000 38.0000 0.0408
level       0.5051 2.0000 76.0000 0.6055
time:level  0.6064 2.0000 76.0000 0.5479



In [31]:
apl.columns[5:]

Index(['Respiratory rate (breaths/min)', 'Heart rate (beats/min)',
       'Ventilation (l/min)', 'O2 intake (l/min)',
       'Relative O2 intake (ml/kg/min)', 'CO2 expired (l/min)',
       'Relative CO2 expired (ml/kg/min)', 'Respiratory exchange ratio',
       'Ventilation oxygen equivalent (l)', 'O2 per pulse (l)',
       'Fat energy (kcal/h)', 'Carbohydrates energy (kcal/h)', 'Fat energy %',
       'Carbohydrates energy %', 'Total energy (kcal/h)',
       'Metabolic equivalent of task (MET) (3.5ml/kg/min)'],
      dtype='object')

In [32]:
apl[['ID', 'Effort level', 'Time',
     'Respiratory rate (breaths/min)']].pivot(columns=['Effort level', 'Time'],
                                              values='Respiratory rate (breaths/min)',
                                              index='ID').agg(['count',
                                                               'median', 'mean',
                                                               'min', 'max', 'std', 'var',
    lambda x: st.kurtosis(x.copy()),
    lambda y: st.skew(y.copy()),
    lambda z: st.shapiro(z)[0],
    lambda z: st.shapiro(z)[1]]).round(2).sort_index(axis=1,
                                                     level=['Effort level', 'Time'],
                                                     ascending=[True,False])

Effort level       1              2              3       
Time         initial  final initial  final initial  final
count          39.00  39.00   39.00  39.00   39.00  39.00
median         30.78  25.16   31.87  27.38   32.68  27.92
mean           31.63  26.49   33.58  28.22   33.68  29.06
min            10.20  14.57   16.40  13.89    0.00  13.24
max            64.44  51.38   62.23  48.25   66.58  56.55
std             9.95   7.99    9.02   8.06   12.04   8.72
var            98.96  63.83   81.41  65.00  144.89  76.08
<lambda>        2.75   1.32    1.07  -0.49    1.90   1.20
<lambda>        1.19   1.12    0.68   0.30    0.40   0.73
<lambda>        0.91   0.92    0.97   0.98    0.93   0.95
<lambda>        0.00   0.01    0.28   0.64    0.02   0.08

In [33]:
for col in apl.columns[20:]:
    print('\n', col, '\n',
    apl[['ID', 'Effort level', 'Time', col]].pivot(columns=['Effort level', 'Time'],
                                                   values=col,
                                                   index='ID').agg([
    'count', 'median', 'mean',
    'min', 'max', 'std', 'var',
    lambda x: st.kurtosis(x.copy()),
    lambda y: st.skew(y.copy()),
    lambda z: st.shapiro(z)[0],
    lambda z: st.shapiro(z)[1]]).sort_index(axis=1,
                                            level=['Effort level', 'Time'],
                                            ascending=[True,False]).round(2))
#.to_csv(f'{col.replace("/"," ")}.csv')


 Metabolic equivalent of task (MET) (3.5ml/kg/min) 
 Effort level       1              2              3       
Time         initial  final initial  final initial  final
count          39.00  39.00   39.00  39.00   39.00  39.00
median          4.38   4.19    4.44   4.17    4.55   4.07
mean            4.52   4.13    4.61   4.25    4.71   4.14
min             2.45   2.27    2.49   2.50    0.00   2.44
max             8.92   6.42    7.23   5.89    8.84   6.35
std             1.24   1.01    1.14   0.90    1.54   0.94
var             1.55   1.01    1.29   0.80    2.37   0.88
<lambda>        2.33  -0.29   -0.56  -0.82    1.69  -0.45
<lambda>        1.18   0.22    0.25   0.04   -0.02   0.18
<lambda>        0.93   0.98    0.98   0.98    0.96   0.99
<lambda>        0.02   0.85    0.65   0.59    0.13   0.92


In [73]:
# for param in aml.parameter.unique():
#     print('\n', param)
#     for lev in range(1,4):
#         print(lev)
#         t1 = aml.groupby(['parameter', 'Effort level', 'Time']).get_group((param, lev, 'initial'))
#         t2 = aml.groupby(['parameter', 'Effort level', 'Time']).get_group((param, lev, 'final'))
#         print(st.ttest_rel(t1.value, t2.value))

In [117]:
lev_ttest_res = pd.DataFrame(data=9,
                             index=range(48),
                             columns=['Parameter', 'Effort level',
                                      'Significance level',
                                      'Degrees of freedom',
                                      'T critical (two tailed)',
                                      'T statistic', 'T test p-value (two-tailed)',
                                      'Levene statistic', 'Levene p-value',
                                      'Effect size (Cohen d)'])
lev_ttest_res['Significance level'] = 0.05

In [168]:
k = 0
for i, j in aml.groupby(['parameter', 'Effort level'], sort=False):
    t_pair = j.pivot(columns='Time', values='value', index='ID').reset_index()
    t_res = st.ttest_rel(t_pair.initial, t_pair.final, alternative='two-sided')
    levene = st.levene(t_pair.initial, t_pair.final)
    cohen = cohen_glass_hedges(t_pair.initial, t_pair.final)
    lev_ttest_res.loc[k, 'Parameter'] = i[0]
    lev_ttest_res.loc[k, 'Effort level'] = i[1]
    lev_ttest_res.loc[k, 'Degrees of freedom'] = len(t_pair.initial) - 1
    lev_ttest_res.loc[k, 'T critical (two tailed)'] = st.t.ppf(0.025, len(t_pair.initial) - 1).round(2)
    lev_ttest_res.loc[k, 'T statistic'] = t_res[0].round(2)
    lev_ttest_res.loc[k, 'T test p-value (two-tailed)'] = t_res[1]
    lev_ttest_res.loc[k, 'Levene statistic'] = levene[0].round(2)
    lev_ttest_res.loc[k, 'Levene p-value'] = levene[0]
    lev_ttest_res.loc[k, 'Effect size (Cohen d)'] = levene[0].round(2)
    k += 1
    #print('\n', i[0], 'Level ', i[1], '\n', st.ttest_rel(t3.initial, t3.final))
results_df = lev_ttest_res.pivot(columns='Effort level',
                                 index='Parameter',
                                 values=['Significance level',
                                         'Degrees of freedom',
                                         'T critical (two tailed)',
                                         'T statistic', 'T test p-value (two-tailed)',
                                         'Levene statistic', 'Levene p-value',
                                         'Effect size (Cohen d)']).reset_index().T

In [170]:
results_df.to_csv('ttresults.csv')

In [179]:
pd.read_csv('ttresults.csv', index_col=['Statistic', 'Effort level']).to_excel('ttresults.xlsx')

In [180]:
results_df

0   \
                            Effort level                        
Parameter                                 CO2 expired (l/min)   
Significance level          1                            0.05   
                            2                            0.05   
                            3                            0.05   
Degrees of freedom          1                            38.0   
                            2                            38.0   
                            3                            38.0   
T critical (two tailed)     1                           -2.02   
                            2                           -2.02   
                            3                           -2.02   
T statistic                 1                            2.95   
                            2                             2.5   
                            3                            2.91   
T test p-value (two-tailed) 1                        0.005412   
                            2                        0.016901   
                            3                        0.006036   
Levene statistic            1                             0.9   
                            2                            1.66   
                            3                            1.88   
Levene p-value              1                        0.896071   
                            2                        1.664218   
                            3                        1.880944   
Effect size (Cohen d)       1                             0.9   
                            2                            1.66   
                            3                            1.88   

                                                              1   \
                            Effort level                           
Parameter                                 Carbohydrates energy %   
Significance level          1                               0.05   
                            2                               0.05   
                            3                               0.05   
Degrees of freedom          1                               38.0   
                            2                               38.0   
                            3                               38.0   
T critical (two tailed)     1                              -2.02   
                            2                              -2.02   
                            3                              -2.02   
T statistic                 1                               2.54   
                            2                                0.9   
                            3                               0.72   
T test p-value (two-tailed) 1                           0.015426   
                            2                           0.372748   
                            3                            0.47879   
Levene statistic            1                               6.26   
                            2                               0.74   
                            3                               0.54   
Levene p-value              1                           6.258887   
                            2                           0.743892   
                            3                           0.536568   
Effect size (Cohen d)       1                               6.26   
                            2                               0.74   
                            3                               0.54   

                                                                     2   \
                            Effort level                                  
Parameter                                 Carbohydrates energy (kcal/h)   
Significance level          1                                      0.05   
                            2                                      0.05   
                            3                                      0.05   
Degrees of freed